In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('data/raw/spx_long.csv')

In [ ]:
print(data.columns)

In [ ]:
#Conduct the same preprocessing steps for the long term option data, should be in surface.ipynb

data['strike_price'] = data['strike_price']/1000

In [ ]:
#Drop unique columns, columns where there are only 1 value possible (no additional info)
drop = []
for i in data.columns:
    print(i)
    print(data[i].unique())

    if len(data[i].unique()) == 1:
        drop.append(i)

data = data.drop(columns=drop)

In [ ]:
data['date'] = pd.to_datetime(data['date'])
data['exdate'] = pd.to_datetime(data['exdate'])

data['maturity'] = np.busday_count(data['date'].values.astype('datetime64[D]'), data['exdate'].values.astype('datetime64[D]'))

In [ ]:
spx = pd.read_csv('spx_prices_daily.csv')
spx['date'] = pd.to_datetime(spx['Date'], format='%d/%m/%Y')
spx = spx.drop(columns='Date')

In [ ]:
data = pd.merge(data, spx, on='date', how='left')

In [ ]:
data['moneyness'] = data['Adj Close'] / data['strike_price']

In [ ]:
first_date = data['date'].min()
last_date = data['date'].max()

print("First Date:", first_date)
print("Last Date:", last_date)

In [ ]:
calls = data[data['cp_flag']=='C']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


def surface(date, option):
    # Pivot the DataFrame

    option_df = data[data['cp_flag']==option]
    option_df = option_df[option_df['date']==date]

    #print(option_df)
    surface = (
        option_df[['maturity', 'moneyness', 'impl_volatility']]
        .pivot_table(values='impl_volatility', index='moneyness', columns='maturity')
        .dropna()
    )

    # Create the figure object
    fig = plt.figure(figsize=(10, 8))

    # Add the subplot with projection argument
    ax = fig.add_subplot(111, projection='3d')

    # Get the 1D values from the pivoted DataFrame
    x, y = surface.columns.values, surface.index.values
    z = surface.values

    # Return coordinate matrices from coordinate vectors
    X, Y = np.meshgrid(x, y)

    # Set labels
    if option == 'C':
        text = 'Call'
    elif option == 'P':
        text = 'Put'
    else:
        text = 'Error'

    ax.set_xlabel('Moneyness')
    ax.set_ylabel('Days to expiration')
  
    ax.set_title(text + ' Implied Volatility Surface')

    # Plot with color map based on IV values
    surf = ax.plot_surface(Y, X, z, cmap='viridis')  # You can choose a different colormap if desired

    # Add a color bar for reference
    fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, label='Implied Volatility')

    # Show the plot
    plt.show()

surface('2023-02-10', 'C')
surface('2023-02-10', 'P')

In [ ]:
# Plot them side by side 
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


def combined_surface(date):
    # Filter data for call options
    call_df = data[(data['cp_flag'] == 'C') & (data['date'] == date)]
    call_surface = (
        call_df[['maturity', 'moneyness', 'impl_volatility']]
        .pivot_table(values='impl_volatility', index='moneyness', columns='maturity')
        .dropna()
    )

    # Filter data for put options
    put_df = data[(data['cp_flag'] == 'P') & (data['date'] == date)]
    put_surface = (
        put_df[['maturity', 'moneyness', 'impl_volatility']]
        .pivot_table(values='impl_volatility', index='moneyness', columns='maturity')
        .dropna()
    )

    fig, (ax1, ax2) = plt.subplots(1, 2, subplot_kw={'projection': '3d'}, figsize=(14, 8))

    # Get the meshgrid values for calls
    x1, y1 = call_surface.columns.values, call_surface.index.values
    z1 = call_surface.values
    X1, Y1 = np.meshgrid(x1, y1)

    # Get the meshgrid values for puts
    x2, y2 = put_surface.columns.values, put_surface.index.values
    z2 = put_surface.values
    X2, Y2 = np.meshgrid(x2, y2)

    color = 'viridis'
    # Plot the call surface
    surf1 = ax1.plot_surface(Y1, X1, z1, cmap=color)
    ax1.set_title('Call Implied Volatility Surface')
    ax1.set_xlabel('Moneyness')
    ax1.set_ylabel('Days to Expiration')
    ax1.set_zlabel('Implied Volatility')

    # Plot the put surface
    surf2 = ax2.plot_surface(Y2, X2, z2, cmap=color)
    ax2.set_title('Put Implied Volatility Surface')
    ax2.set_xlabel('Moneyness')
    ax2.set_ylabel('Days to Expiration')
    ax2.set_zlabel('Implied Volatility')

    # Add a single color bar for both subplots
    fig.colorbar(surf2, ax=[ax1, ax2], shrink=0.5, aspect=10, pad=0.1, label='Implied Volatility')

    # Show the plot
    plt.show()

# Call the function to plot for a specific date and option types
combined_surface('2023-02-10')


In [ ]:
print(data[(data['date']=='2023-02-10') & (data['cp_flag'] == 'C')]['maturity'].unique())

In [ ]:
data_adj = data.copy()

In [ ]:
data_adj.loc[data_adj['am_settlement']==1, 'maturity'] -=1

In [ ]:
# PREPROCESS
data_adj['midpoint'] = ( data_adj['best_bid'] + data_adj['best_offer'] ) /2 

data_adj = data_adj[data_adj['volume']!= 0]
data_adj = data_adj[data_adj['midpoint'] > 0.125]
data_adj = data_adj[data_adj['best_bid'] != 0]
data_adj = data_adj.dropna(subset='impl_volatility')

In [ ]:
data_adj.to_csv("data_adj_long.csv", index=False)

In [ ]:
test= pd.read_csv('data_adj_long.csv')

In [ ]:
print(test.shape)

In [ ]:
print(test.columns)

In [ ]:
print(test['impl_volatility'])